# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime
import os
import sys

import mujoco
import mediapy as media
from gym import Env
from gym.spaces import Discrete, Box

import stable_baselines3
from stable_baselines3.common.env_checker import check_env
import torch as th
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.logger import configure
from stable_baselines3.common.logger import Logger, CSVOutputFormat, HumanOutputFormat

import torch
import torch.nn as nn


from stable_baselines3.common.callbacks import BaseCallback
from torch.utils.data import DataLoader, TensorDataset

# Cargar los datos de captura de movimiento y simulación

In [2]:
datos = pd.read_csv('datos_seguimiento_3_polyfit15_conpos_g1.csv',sep=",")
array_datos = datos.to_numpy() #guarda los datos del csv cargado como arreglo de numpy

with open("g1_29dof_position_arnesTUBO16.xml", "r", encoding="utf-8") as f:
    xml = f.read() #Leer el archivo de mujoco cargado al entorno de ejecución y lo guarda en la variable xml

In [ ]:
#Discriminador en PyTorch
class AMPDiscriminator(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# Ambiente de Gym

In [ ]:
#Se Modifica el entorno para usar el discriminador
#Se cambia step() para usar el discriminador:


class RobotBipedoG1(Env):
  def __init__(self, xml_text, tracking_array, tiempo_max_sim, tracking_fps=30):

    #Algunos parámetros asociados a la física del robot y la simulación de MuJoCo
    self.simulation_time_step=0.002 #La simulación corre en pasos de 0.002 segundos (frecuencia de 500 Hz)
    self.max_step_motors=0.003 #Delta de Posición


    #Se crea la simulación con el xml cargado
    # Make model and data
    self.model = mujoco.MjModel.from_xml_string(xml_text)
    self.data = mujoco.MjData(self.model)


    # El estado es definido por 8 valores
    # Orden: 
    # 0 Angulo Cadera-Rodilla pierna 1
    # 1 Angulo Rodilla-Tobillo pierna 1
    # 2 Angulo Cadera-Tobillo pierna 1
    # 3 Angulo Cadera-Rodilla pierna 2 
    # 4 Angulo Rodilla-Tobillo pierna 2
    # 5 Angulo Cadera-Tobillo pierna 2
    # 6 tiempo
    self.observation_space = Box(low=-np.inf, high=np.inf, shape=(19, ))

    # Las acciones son definidas como un continuo en cada paso de la simulación. Limitado por el paso de la simulación y velocidad motores
    # Orden: 
    # 0 right_hip_pitch_joint
    # 1 right_hip_roll_joint
    # 2 right_hip_yaw_joint
    # 3 right_knee_joint 
    # 4 right_ankle_pitch_joint
    # 5 right_ankle_roll_joint
    # 6 left_hip_pitch_joint
    # 7 left_hip_roll_joint
    # 8 left_hip_yaw_joint
    # 9 left_knee_joint 
    # 10 left_ankle_pitch_joint
    # 11 left_ankle_roll_joint
    # 12 waist_pitch_joint
    self.action_space = Box(low=-1, high=1, shape=(12,))

    #El estado inicial está dado por el tracking inicial
    # 0  Cadera pierna 1 x
    # 1  Cadera pierna 1 y
    # 2  Cadera pierna 1 z
    # 3  Rodilla pierna 1 x
    # 4  Rodilla pierna 1 y
    # 5  Rodilla pierna 1 z
    # 6  Tobillo pierna 1 x
    # 7  Tobillo pierna 1 y
    # 8  Tobillo pierna 1 z
    # 9  Cadera pierna 2 x
    # 10 Cadera pierna 2 y
    # 11 Cadera pierna 2 z
    # 12 Rodilla pierna 2 x
    # 13 Rodilla pierna 2 y
    # 14 Rodilla pierna 2 z
    # 15 Tobillo pierna 2 x
    # 16 Tobillo pierna 2 y
    # 17 Tobillo pierna 2 z
    # 18 tiempo
    self.state=np.concatenate((self.data.geom('right_hip_pitch_link_tracker').xpos,
                              self.data.geom('right_knee_link_tracker').xpos,
                              self.data.geom('right_ankle_pitch_link_tracker').xpos,
                              self.data.geom('left_hip_pitch_link_tracker').xpos,
                              self.data.geom('left_knee_link_tracker').xpos,
                              self.data.geom('left_ankle_pitch_link_tracker').xpos,
                              np.array([self.data.time])))
    

    #Se configura un tiempo final de simulación
    if(tiempo_max_sim<=4.9):
      self.final_simulation_time=tiempo_max_sim
    else:
      self.final_simulation_time=4.9  #la simulación se termina en 5 segundos

    #Se almacena la secuencia de tracking de movimiento a seguir
    #Orden por columnas: Número de frame a X Hz, Ángulo cadera-rodilla_p1, Ángulo rodilla-tobillo_p1, Ángulo cadera-tobillo_p1, Ángulo cadera-rodilla_p2, Ángulo rodilla-tobillo_p2, Ángulo cadera-tobillo_p2 
    self.framerate=tracking_fps
    self.tracking=tracking_array

    #Crea una lista vacía para guardar frames de simulación (60 Hz) y otras opciones
    self.height=480
    self.width=640
    self.frames=[]
    self.renderer = mujoco.Renderer(self.model, self.height, self.width)

    #Inicializa el primer objetivo
    self.obj_frame=1
    self.objective_pos=self.tracking[self.obj_frame-1][7]



    #Iniciliza Discriminador
    self.discriminator = AMPDiscriminator(input_dim=7)
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.discriminator.to(self.device)
    print("¿AMP usando cpu o gpu?")
    print("Utilizando "+str(self.device)+" para entrenar AMP.")
    self.amp_rew_prop=0.125
    print("AMP Reward: "+str(self.amp_rew_prop*100)+"%")
    self.agent_buffer = []
    #Carga estados reales del CSV
    self.real_data = tracking_array[:, 1:8]




  def map_action(self, action_normalized):
    if action_normalized<-0.5:
      return -self.max_step_motors
    elif action_normalized>0.5:
      return self.max_step_motors
    else:
      return 0.000

  #Función donde se accionan los controladores, según la acción y los límites
  def actuate_model(self, action):
    #Añade a los controladores los ángulos del vector de acciones
    self.data.actuator("right_hip_pitch_joint").ctrl+= self.map_action(action[0])
    self.data.actuator("right_hip_roll_joint").ctrl+= self.map_action(action[1])
    self.data.actuator("right_hip_yaw_joint").ctrl+= self.map_action(action[2])
    self.data.actuator("right_knee_joint").ctrl+= self.map_action(action[3])
    self.data.actuator("right_ankle_pitch_joint").ctrl+= self.map_action(action[4])
    self.data.actuator("right_ankle_roll_joint").ctrl+= self.map_action(action[5])
    self.data.actuator("left_hip_pitch_joint").ctrl+= self.map_action(action[6])
    self.data.actuator("left_hip_roll_joint").ctrl+= self.map_action(action[7])
    self.data.actuator("left_hip_yaw_joint").ctrl+= self.map_action(action[8])
    self.data.actuator("left_knee_joint").ctrl+= self.map_action(action[9])
    self.data.actuator("left_ankle_pitch_joint").ctrl+= self.map_action(action[10])
    self.data.actuator("left_ankle_roll_joint").ctrl+= self.map_action(action[11])
    """
    self.data.actuator("right_hip_pitch_joint").ctrl+=np.clip(action[0],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("right_hip_roll_joint").ctrl+=np.clip(action[1],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("right_hip_yaw_joint").ctrl+=np.clip(action[2],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("right_knee_joint").ctrl+=np.clip(action[3],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("right_ankle_pitch_joint").ctrl+=np.clip(action[4],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("right_ankle_roll_joint").ctrl+=np.clip(action[5],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("left_hip_pitch_joint").ctrl+=np.clip(action[6],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("left_hip_roll_joint").ctrl+=np.clip(action[7],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("left_hip_yaw_joint").ctrl+=np.clip(action[8],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("left_knee_joint").ctrl+=np.clip(action[9],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("left_ankle_pitch_joint").ctrl+=np.clip(action[10],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("left_ankle_roll_joint").ctrl+=np.clip(action[11],-self.max_step_motors,self.max_step_motors)"""
    #self.data.actuator("waist_pitch_joint").ctrl+=np.clip(action[12],-self.max_step_motors,self.max_step_motors)
    
    self.data.actuator("arnes1").ctrl=self.objective_pos+0.05

    #Usa clip para aplicar los límites superiores e inferiores
    self.data.actuator("right_hip_pitch_joint").ctrl=np.clip(self.data.actuator("right_hip_pitch_joint").ctrl,-2.5307,2.8798)
    self.data.actuator("right_hip_roll_joint").ctrl=np.clip(self.data.actuator("right_hip_roll_joint").ctrl,-0.5236,2.9671)
    self.data.actuator("right_hip_yaw_joint").ctrl=np.clip(self.data.actuator("right_hip_yaw_joint").ctrl,-2.7576,2.7576)
    self.data.actuator("right_knee_joint").ctrl=np.clip(self.data.actuator("right_knee_joint").ctrl,-0.087267,2.8798)
    self.data.actuator("right_ankle_pitch_joint").ctrl=np.clip(self.data.actuator("right_ankle_pitch_joint").ctrl,-0.87267,0.5236)
    self.data.actuator("right_ankle_roll_joint").ctrl=np.clip(self.data.actuator("right_ankle_roll_joint").ctrl,-0.2618,0.2618)
    self.data.actuator("left_hip_pitch_joint").ctrl=np.clip(self.data.actuator("left_hip_pitch_joint").ctrl,-2.5307,2.8798)
    self.data.actuator("left_hip_roll_joint").ctrl=np.clip(self.data.actuator("left_hip_roll_joint").ctrl,-2.9671,0.5236)
    self.data.actuator("left_hip_yaw_joint").ctrl=np.clip(self.data.actuator("left_hip_yaw_joint").ctrl,-2.7576,2.7576)
    self.data.actuator("left_knee_joint").ctrl=np.clip(self.data.actuator("left_knee_joint").ctrl,-0.087267,2.8798)
    self.data.actuator("left_ankle_pitch_joint").ctrl=np.clip(self.data.actuator("left_ankle_pitch_joint").ctrl,-0.87267,0.5236)
    self.data.actuator("left_ankle_roll_joint").ctrl=np.clip(self.data.actuator("left_ankle_roll_joint").ctrl,-0.2618,0.2618)
    #self.data.actuator("waist_pitch_joint").ctrl=np.clip(self.data.actuator("waist_pitch_joint").ctrl,-0.52,0.52)

    self.data.actuator("arnes1").ctrl=np.clip(self.data.actuator("arnes1").ctrl,-1,3)
    
  

  def step(self,action):

    #Ejecuta la acción y se cambian los controladores en la simulación
    self.actuate_model(action)
    #Da el paso de simulación
    mujoco.mj_step(self.model, self.data) #necesita dar un paso al inicio
    #Lee las nuevas posiciones (estado)

    

    self.state=np.concatenate((self.data.geom('right_hip_pitch_link_tracker').xpos,
                              self.data.geom('right_knee_link_tracker').xpos,
                              self.data.geom('right_ankle_pitch_link_tracker').xpos,
                              self.data.geom('left_hip_pitch_link_tracker').xpos,
                              self.data.geom('left_knee_link_tracker').xpos,
                              self.data.geom('left_ankle_pitch_link_tracker').xpos,
                              np.array([self.data.time])))
    
    current_angle_cadera_rodilla_pierna1=np.arctan(((self.state[3]-self.state[0]))/np.abs((self.state[5]-self.state[2])))
    current_angle_rodilla_tobillo_pierna1=np.arctan(((self.state[6]-self.state[3]))/np.abs((self.state[8]-self.state[5])))
    current_angle_cadera_tobillo_pierna1=np.arctan(((self.state[6]-self.state[0]))/np.abs((self.state[8]-self.state[2])))
    current_angle_cadera_rodilla_pierna2=np.arctan(((self.state[12]-self.state[9]))/np.abs((self.state[14]-self.state[11])))
    current_angle_rodilla_tobillo_pierna2=np.arctan(((self.state[15]-self.state[12]))/np.abs((self.state[17]-self.state[14])))
    current_angle_cadera_tobillo_pierna2=np.arctan(((self.state[15]-self.state[9]))/np.abs((self.state[17]-self.state[11])))
    current_pos=(self.state[0]+self.state[9])/2
    current_align_angle_rodilla_tobillo_pierna1=np.arctan(((self.state[7]-self.state[4]))/np.abs((self.state[8]-self.state[5])))
    current_align_angle_cadera_tobillo_pierna1=np.arctan(((self.state[7]-self.state[1]))/np.abs((self.state[8]-self.state[2])))
    current_align_angle_rodilla_pierna1=np.arctan(((self.data.geom('right_knee_link_align_tracker').xpos[1]-self.state[4]))/np.abs((self.data.geom('right_knee_link_align_tracker').xpos[0]-self.state[3])))
    current_align_angle_rodilla_tobillo_pierna2=np.arctan(((self.state[16]-self.state[13]))/np.abs((self.state[17]-self.state[14])))
    current_align_angle_cadera_tobillo_pierna2=np.arctan(((self.state[16]-self.state[10]))/np.abs((self.state[17]-self.state[11])))
    current_align_angle_rodilla_pierna2=np.arctan(((self.data.geom('left_knee_link_align_tracker').xpos[1]-self.state[13]))/np.abs((self.data.geom('left_knee_link_align_tracker').xpos[0]-self.state[12])))

    #Calcula una reward, tiempo de simulación y objetivo de tracking de movimiento en frame específico
    #revisa el tiempo actual de simulación, contraste con los objetivos de la captura de movimiento, y revisa los ángulos objetivo
    if(self.data.time > self.obj_frame/self.framerate):
      self.obj_frame+=1
    objective_angle_cadera_rodilla_pierna1=self.tracking[self.obj_frame-1][1]
    objective_angle_rodilla_tobillo_pierna1=self.tracking[self.obj_frame-1][2]
    objective_angle_cadera_tobillo_pierna1=self.tracking[self.obj_frame-1][3]
    objective_angle_cadera_rodilla_pierna2=self.tracking[self.obj_frame-1][4]
    objective_angle_rodilla_tobillo_pierna2=self.tracking[self.obj_frame-1][5]
    objective_angle_cadera_tobillo_pierna2=self.tracking[self.obj_frame-1][6]
    self.objective_pos=self.tracking[self.obj_frame-1][7] 
    objective_align_angle_rodilla_tobillo_pierna1=0
    objective_align_angle_cadera_tobillo_pierna1=0
    objective_align_angle_rodilla_pierna1=0
    objective_align_angle_rodilla_tobillo_pierna2=0
    objective_align_angle_cadera_tobillo_pierna2=0
    objective_align_angle_rodilla_pierna2=0

    

    #calcula reward
    rp=np.exp(-2*((objective_angle_cadera_rodilla_pierna1-current_angle_cadera_rodilla_pierna1)**2+(objective_angle_rodilla_tobillo_pierna1-current_angle_rodilla_tobillo_pierna1)**2+(objective_angle_cadera_rodilla_pierna2-current_angle_cadera_rodilla_pierna2)**2+(objective_angle_rodilla_tobillo_pierna2-current_angle_rodilla_tobillo_pierna2)**2+(objective_align_angle_rodilla_tobillo_pierna1-current_align_angle_rodilla_tobillo_pierna1)**2+(objective_align_angle_rodilla_tobillo_pierna2-current_align_angle_rodilla_tobillo_pierna2)**2+(objective_align_angle_rodilla_pierna1-current_align_angle_rodilla_pierna1)**2+(objective_align_angle_rodilla_pierna2-current_align_angle_rodilla_pierna2)**2))
    rv=0
    re=np.exp(-40*((objective_angle_cadera_tobillo_pierna1-current_angle_cadera_tobillo_pierna1)**2+(objective_angle_cadera_tobillo_pierna2-current_angle_cadera_tobillo_pierna2)**2+(objective_align_angle_cadera_tobillo_pierna1-current_align_angle_cadera_tobillo_pierna1)**2+(objective_align_angle_cadera_tobillo_pierna2-current_align_angle_cadera_tobillo_pierna2)**2))
    rc=np.exp(-10*((self.objective_pos-current_pos)**2))
    env_reward=0.65*rp+0.1*rv+0.15*re+0.1*rc

    #Condiciones de finalización
    #si pierde el equilibrio y se cae
    done=False
    if(self.state[2]<0.44 or self.state[11]<0.44):
      done=True
      reward=-80
    if(self.data.time >= self.final_simulation_time):
      done=True

    #Complementar
    info={
      "reward": env_reward,
      "state": self.state,
      "objective": self.tracking[self.obj_frame-1],
      "time": self.data.time
    }



    #Guarda una observación para entrenar el discriminador más adelante
    joint_angles = np.array([current_angle_cadera_rodilla_pierna1,
                              current_angle_rodilla_tobillo_pierna1,
                              current_angle_cadera_tobillo_pierna1,
                              current_angle_cadera_rodilla_pierna2,
                              current_angle_rodilla_tobillo_pierna2,
                              current_angle_cadera_tobillo_pierna2,
                              current_pos])
    
    #print("1")
    self.agent_buffer.append(joint_angles.copy())
    #print("2")
    #Calcula recompensa AMP
    with torch.no_grad():
        amp_input = torch.from_numpy(joint_angles).float().unsqueeze(0).to(self.device)
        amp_reward = self.discriminator(amp_input).item()
    #print("3")
    
    reward = (1-self.amp_rew_prop) * env_reward + self.amp_rew_prop * (0.9*amp_reward)






    #Retornos
    return self.state, reward, done, info


  def render(self):
    #después de la función step(action) se llama a esta función, así que se comentan algunas líneas
    if len(self.frames) < self.data.time * self.framerate:
      self.renderer.update_scene(self.data)
      pixels = self.renderer.render()
      self.frames.append(pixels)

  def reset(self):
    self.frames=[] #borra los frames
    self.obj_frame=1 #reinicia el objetivo de tracking
    mujoco.mj_resetData(self.model, self.data) #reinicia la simulación
    
    self.state==np.concatenate((self.data.geom('right_hip_pitch_link_tracker').xpos,
                              self.data.geom('right_knee_link_tracker').xpos,
                              self.data.geom('right_ankle_pitch_link_tracker').xpos,
                              self.data.geom('left_hip_pitch_link_tracker').xpos,
                              self.data.geom('left_knee_link_tracker').xpos,
                              self.data.geom('left_ankle_pitch_link_tracker').xpos,
                              np.array([self.data.time])))
    
    self.objective_pos=self.tracking[self.obj_frame-1][7]

    return self.state


In [ ]:
# Se Entrena el discriminador con un Callback cada n pasos


class AMPCallback(BaseCallback):
    def __init__(self, env, train_freq=2048, batch_size=128, **kwargs):
        super().__init__(**kwargs)
        self.env = env
        self.train_freq = train_freq
        self.batch_size = batch_size
        self.disc_train_step = 0  #un contador de monitoreo para debugging

    def _on_step(self):
        if self.n_calls % self.train_freq == 0:
            self.train_discriminator()
        return True

    def train_discriminator(self):
        # Datos del agente
        fake = np.array(self.env.agent_buffer)
        self.env.agent_buffer = []  # Limpia el buffer
        
        # Datos reales
        real = self.env.real_data[np.random.choice(len(self.env.real_data), len(fake))]
        
        # Tensors
        fake_t = torch.tensor(fake, dtype=torch.float32).to(self.env.device)
        real_t = torch.tensor(real, dtype=torch.float32).to(self.env.device)
        labels_fake = torch.zeros((len(fake), 1), device=self.env.device)
        labels_real = torch.ones((len(real), 1), device=self.env.device)

        # Mezclar
        x = torch.cat([real_t, fake_t])
        y = torch.cat([labels_real, labels_fake])

        dataset = TensorDataset(x, y)
        loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

        # Entrenamiento
        self.env.discriminator.train()
        opt = torch.optim.Adam(self.env.discriminator.parameters(), lr=1e-4)
        loss_fn = nn.BCELoss()

        for batch_x, batch_y in loader:
            opt.zero_grad()
            pred = self.env.discriminator(batch_x)
            loss = loss_fn(pred, batch_y)
            loss.backward()
            opt.step()
            self.disc_train_step += 1
            # Debugging
            #if self.disc_train_step % 100 == 0:
            #    print(f"[Discriminator] Step: {self.disc_train_step} | Loss: {loss.item():.4f}")

In [6]:
env=RobotBipedoG1(xml_text=xml, tracking_array=array_datos, tiempo_max_sim=2, tracking_fps=30)
#el ambiente es creado usando la variable xml donde se cargó la simulación y el arreglo con la captura de movimiento

#Crea el callback
callback = AMPCallback(env)

#crea el modleo normal
policy_kwargs = dict(activation_fn=th.nn.ReLU,net_arch=dict(pi=[1024,512,256,128], vf=[1024,512,256,128]))
model = PPO(
        "MlpPolicy", env,
        learning_rate=0.0001,
        n_steps=4096,
        batch_size=1024,
        n_epochs=10,
        gamma=0.99,
        gae_lambda=0.99,
        clip_range=0.075,
        ent_coef=0.005,
        policy_kwargs=policy_kwargs,
        verbose=0
    )

# crea el logger
# Crear el nombre del archivo CSV con timestamp
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M')
csv_path = f"./logs/ppo_amp1_{timestamp}.csv"
# Crear un nuevo logger con CSV 
new_logger = Logger(folder="./logs/",output_formats=[CSVOutputFormat(csv_path)])
# Asignar el nuevo logger al modelo
model.set_logger(new_logger)

#Entrena con callback
model.learn(total_timesteps=5_000_000, progress_bar=True, callback=callback)

Output()

¿AMP usando cpu o gpu?
Utilizando cuda para entrenar AMP.
AMP Reward: 12.5%


c:\Users\ssola_txrzbqq\AppData\Local\Programs\Python\Python38\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
c:\Users\ssola_txrzbqq\AppData\Local\Programs\Python\Python38\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(

In [7]:
model.save(f"ppo_amp1_{timestamp}")
print(f"model saved as ppo_amp1_{timestamp}")

model saved as ppo_amp1_2025-06-19_07-49


In [8]:
eval_env=RobotBipedoG1(xml_text=xml, tracking_array=array_datos, tiempo_max_sim=2)
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=5)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

¿AMP usando cpu o gpu?
Utilizando cuda para entrenar AMP.
AMP Reward: 12.5%


c:\Users\ssola_txrzbqq\AppData\Local\Programs\Python\Python38\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:332.16 +/- 0.89


In [9]:
episodios=1
for i in range(0,episodios):
  state=eval_env.reset()
  new_state=state
  done=False
  recompensa_acumulada=0
  pasos=0

  while not done:
    state=new_state
    pasos+=1
    action, _states = model.predict(state, deterministic=True)
    new_state,reward,done,info=eval_env.step(action)
    recompensa_acumulada+=reward
  print("Episodio {}: Recompensa total {} en {} pasos de tiempo.".format(i+1,recompensa_acumulada,pasos))

Episodio 1: Recompensa total 331.7110478620603 en 1000 pasos de tiempo.


In [ ]:
state=eval_env.reset()
new_state=state
done=False
recompensa_acumulada=0
pasos=0

while not done:
  state=new_state
  pasos+=1
  action, _states = model.predict(state, deterministic=True)
  #print(action)
  new_state,reward,done,info=eval_env.step(action)
  #print(new_state[1])
  recompensa_acumulada+=reward
  print(reward)
  eval_env.render()

print("Recompensa total {} en {} pasos de tiempo.".format(recompensa_acumulada,pasos))
media.show_video(eval_env.frames, fps=eval_env.framerate)

0.7435292966557434
0.7435292966557434
0.7435265441084431
0.7435440273686889
0.7436046448333943
0.7437265351833692
0.7439222195600572
0.7440552139628995
0.7439929757458086
0.7438808493106311
0.7438202404910716
0.7438230735660993
0.7438892868063556
0.7440184911061968
0.7442096417434846
0.7444618875715645
0.7267416308252758
0.7269141401635485
0.7271254520130207
0.7273744818157363
0.727660335827371
0.727982218113593
0.7283392500351369
0.7287303396575604
0.7291540017440399
0.7296084195426288
0.7300913209816652
0.7305999382590669
0.7311310035820766
0.7316806540625446
0.7322444280271336
0.7328182007738038
0.7333977093790292
0.7018959210587583
0.7021739312635056
0.7024355714975669
0.7026763755399331
0.7028924777919219
0.7030795409587496
0.7032340905773368
0.7033516276783396
0.7034280914874763
0.7034590801996327
0.7034398699309576
0.7035522092495439
0.7037639759787223
0.7040456199140505
0.7043703631373085
0.704714328468344
0.698166330393023
0.6995071786355965
0.7008254976992487
0.70210511492899

This browser does not support the video tag.


: 